In [5]:
import itertools
import time
import warnings
from functools import partial
import json

import numpy as np
import pandas as pd
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

import datetime
import platform


warnings.filterwarnings("ignore")


def flat(ll):
    return list(itertools.chain(*ll))

Основной класс, который образует маршруты

In [8]:

class Worker:

    def vrp(self):

        location_all = pd.concat([self.dep2] + [self.location],
                                 axis=0)

        # ограничиваем матрицу расстояний до нужных пределов
        dd_matrix = self.d_matrix.loc[location_all.index, location_all.index]

        ###########################
        # Problem Data Definition #
        ###########################
        def create_data_model():
            """Stores the data for the problem"""
            data = {}

            data['distance_matrix'] = dd_matrix.astype(int).values.tolist()
            data['demands'] = list(location_all.loc[:, 'volume'].astype(int))
            data['must'] = list(location_all.loc[:, 'must'].astype(int))

            data['num_locations'] = len(data['demands'])
            data['time_per_demand_unit'] = list(
                location_all.loc[:, 'time_spent'].astype(int))
            data['time_windows'] = [(int(i), int(k)) for i, k in
                                    zip(location_all['time_from'], location_all['time_to'])]
            data['num_vehicles'] = self.num_vehic
            data['vehicle_capacity'] = self.vehicle_capacity
            data['depot'] = 0
            return data

        #######################
        # Problem Constraints #
        #######################

        def create_distance_evaluator(data):
            """Creates callback to return distance between points."""
            _distances = data['distance_matrix']

            def distance_evaluator(manager, from_node, to_node):
                """Returns the manhattan distance between the two nodes"""
                return _distances[manager.IndexToNode(from_node)][manager.IndexToNode(to_node)]
            # - data['demands'][manager.IndexToNode(from_node)]
            #

            return distance_evaluator

        def create_demand_evaluator(data):

            def demand_evaluator(manager, from_node):
                return data['demands'][manager.IndexToNode(from_node)]

            return demand_evaluator

        def add_capacity_constraints(routing, manager, data, demand_evaluator_index):
            capacity = 'Capacity'
            routing.AddDimension(
                demand_evaluator_index,
                0,  # null capacity slack
                data['vehicle_capacity'],  # vehicle maximum capacities
                True,  # start cumul to zero
                'Capacity')

            list_add_junc1 = []
            list_add_junc2 = []
            for node in range(1, len(data['demands'])):
                if data['must'][node] == 1:
                    list_add_junc1 += [node]
                elif data['must'][node] == 2:
                    list_add_junc2 += [node]
                else:
                    pass

            for node in list_add_junc1:
                routing.AddDisjunction([manager.NodeToIndex(node)], 1000_0000)

            for node in list_add_junc2:
                routing.AddDisjunction([manager.NodeToIndex(node)], 1000)

        def create_time_evaluator(data):
            """Creates callback to get total times between locations."""

            def service_time(data, node):
                """Gets the service time for the specified location."""
                return abs(data['time_per_demand_unit'][node])

            def travel_time(data, from_node, to_node):
                """Gets the travel times between two locations."""
                if from_node == to_node:
                    travel_time = 0
                else:
                    travel_time = int(
                        data['distance_matrix'][from_node][to_node])
                return travel_time

            _total_time = {}
            # precompute total time to have time callback in O(1)
            for from_node in range(data['num_locations']):
                _total_time[from_node] = {}
                for to_node in range(data['num_locations']):
                    if from_node == to_node:
                        _total_time[from_node][to_node] = 0
                    else:
                        _total_time[from_node][to_node] = int(
                            service_time(data, from_node) + travel_time(
                                data, from_node, to_node))

            def time_evaluator(manager, from_node, to_node):
                """Returns the total time between the two nodes"""
                return _total_time[manager.IndexToNode(from_node)][manager.IndexToNode(
                    to_node)]

            return time_evaluator

        def add_time_window_constraints(routing, manager, data, time_evaluator):
            """Add Time windows constraint"""
            time = 'Time'
            routing.AddDimension(
                time_evaluator,
                2000,  # allow waiting time
                1600,  # maximum time per vehicle
                False,  # don't force start cumul to zero since we are giving TW to start nodes
                time)
            time_dimension = routing.GetDimensionOrDie(time)
            # Add time window constraints for each location except depot
            # and 'copy' the slack var in the solution object (aka Assignment) to print it
            for location_idx, time_window in enumerate(data['time_windows']):
                if location_idx in [0]:
                    continue
                index = manager.NodeToIndex(location_idx)
                time_dimension.CumulVar(index).SetRange(
                    time_window[0], time_window[1])
                routing.AddToAssignment(time_dimension.SlackVar(index))
            # Add time window constraints for each vehicle start node
            # and 'copy' the slack var in the solution object (aka Assignment) to print it

            from_n = [self.start_time_veh]*data['num_vehicles']
            to_n = [self.end_time_veh]*data['num_vehicles']

            for vehicle_id in range(data['num_vehicles']):
                index = routing.Start(vehicle_id)
                time_dimension.CumulVar(index).SetRange(int(from_n[vehicle_id]),
                                                        int(to_n[vehicle_id]))

                routing.AddToAssignment(time_dimension.SlackVar(index))

                index = routing.End(vehicle_id)
                time_dimension.CumulVar(index).SetRange(int(from_n[vehicle_id]),
                                                        int(to_n[vehicle_id]))

        def print_solution(data, manager, routing, assignment):

            time_dimension = routing.GetDimensionOrDie('Time')

            route = []

            for vehicle_id in range(data['num_vehicles']):

                index = routing.Start(vehicle_id)
                route_time = []
                route_volume = []
                output = []
                route_money = []

                while not routing.IsEnd(index):

                    node_index = manager.IndexToNode(index)

                    route_volume += [data['demands'][node_index]]

                    time_var = time_dimension.CumulVar(index)
                    route_time += [(assignment.Min(time_var),
                                    assignment.Max(time_var))]

                    previous_index = index

                    index = assignment.Value(routing.NextVar(index))

                    if index != -1:  # routing.Start(vehicle_id):
                        route_money += [routing.GetArcCostForVehicle(
                            previous_index, index, vehicle_id)]

                        # dist_var = distance_dimension.CumulVar(index)
                        # route_dist += [assignment.Value(dist_var)]

                    output += [node_index]

                node_index = manager.IndexToNode(index)

                route_volume += [data['demands'][node_index]]

                time_var = time_dimension.CumulVar(index)
                route_time += [(assignment.Min(time_var),
                                assignment.Max(time_var))]

                output += [node_index]

                if len(output) > 2:

                    rinfo = {'uid': location_all.iloc[output].index.tolist(),
                             'dist': [0] + [int(i) for i in route_money]
                             }
                    rinfo['must'] = location_all['must'].iloc[output].tolist()

                    rinfo['time'] = [j[0] for j in route_time]

                    rinfo['volume'] = route_volume[:-1] + [0]

                    rinfo['dist'] += [
                        create_distance_evaluator(data)(manager, 1, 0)]
                    rinfo['time'] += [rinfo['time'][-1] +
                                      create_time_evaluator(data)(manager, 1, 0)]

                    route += [rinfo]

            return route

        """Entry point of the program"""
        # Instantiate the data problem.
        data = create_data_model()

        # Create the routing index manager
        manager = pywrapcp.RoutingIndexManager(
            len(data['distance_matrix']), data['num_vehicles'], 0)

        # Create Routing Model
        routing = pywrapcp.RoutingModel(manager)

        # Define weight of each edge
        distance_evaluator_index = routing.RegisterTransitCallback(
            partial(create_distance_evaluator(data), manager))
        routing.SetArcCostEvaluatorOfAllVehicles(distance_evaluator_index)

        # routing.SetFixedCostOfAllVehicles(self.fixed)

        # Add Capacity constraint
        demand_evaluator_index = routing.RegisterUnaryTransitCallback(
            partial(create_demand_evaluator(data), manager))
        add_capacity_constraints(
            routing, manager, data, demand_evaluator_index)

        # Add Time Window constraint
        time_evaluator_index = routing.RegisterTransitCallback(
            partial(create_time_evaluator(data), manager))
        add_time_window_constraints(
            routing, manager, data, time_evaluator_index)

        # Setting first solution heuristic (cheapest addition).
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

        if platform.system() == 'Windows':
            search_parameters.log_search = True

        # Solve the problem.
        st = time.time()
        solution = routing.SolveWithParameters(search_parameters)

        if solution:
            route = print_solution(data, manager, routing, solution)
        else:
            print("No solution found !")

        ptime = time.time() - st
        print(ptime)
        print(len(route))
        print('dist', [max(i['time']) for i in route])
        print('dist', sum([sum(i['volume']) for i in route]))

        return route


Импорт всех таблиц, уменьшение размера чисел, матрица асстояний как матрица

In [9]:
OptimusG = Worker()

OptimusG.dep2 = pd.DataFrame([{'volume': 0, 'time_from': 480, 'time_to': 1200, 'time_spent': 0, 'must': 0}])

OptimusG.start_time_veh = 480
OptimusG.end_time_veh = 1200
OptimusG.vehicle_capacity = 1000_000

if platform.system() == 'Windows':
    d1 = pd.read_csv('C:/Users/User/Desktop/hac/times v4.csv')
    table_incomes = pd.read_excel(
        'C:/Users/User/Desktop/hac/terminal_data_hackathon v4.xlsx', sheet_name='Incomes').set_index('TID')
else:
    d1 = pd.read_csv('/root/times v4.csv')
    table_incomes = pd.read_excel(
        '/root/terminal_data_hackathon v4.xlsx', sheet_name='Incomes').set_index('TID')

table_incomes['2022-12-01'] = 0
table_incomes = (table_incomes/1000).astype(int)
table_incomes['90%'] = table_incomes.iloc[:, 1:].apply(lambda x: np.percentile(x, 90), axis=1)
table_incomes['25%'] = table_incomes.iloc[:, 1:-1].apply(lambda x: np.percentile(x, 25), axis=1)

all_free_dive = table_incomes.loc[(table_incomes['90%'] < 150)|
                                  (table_incomes['25%'] < 50)].index


# формирование матрицы расстояний
d1 = pd.pivot_table(d1, index='Origin_tid',
                    columns='Destination_tid', values='Total_Time').fillna(0)
d1 = pd.concat([pd.DataFrame([0]*len(d1), index=d1.index).T, d1], axis=0)
d1[0] = 0
OptimusG.d_matrix = np.round(d1[sorted(d1.columns)])


# функция, которая поддает оптимальные точки подлизости
def get_nearnodes(tam, takaa):
    ta_ta = pd.DataFrame()
    for i in tam:
        tadam = OptimusG.d_matrix.loc[[i], takaa].T
        tadam.columns = [0]
        ta_ta = pd.concat([ta_ta, tadam], axis=0)
    ta_ta = ta_ta.reset_index().groupby('index')[[0]].min().sort_values(0)
    return [i for i in ta_ta.index if i not in tam]


Основной цикл, который перебирает во все дни

In [ ]:
da_inc = pd.DataFrame()
routes = []
used = []
for n in range(0, 92):

    if n % 14 == 0:
        used = []

    print('\n\n Count', n)
    m, n1 = table_incomes.columns[n], table_incomes.columns[n+1]
    print(m)

    loct = table_incomes[[m, '90%']]
    loct['must'] = (loct[m] + loct['90%'] > 1000).astype(int)

    little_bb_init = list(loct.loc[loct['must'] == 1].index)
    little_m_have = list(set(all_free_dive) - set(used))

    little_m_have = get_nearnodes(little_bb_init, little_m_have)

    # те точки которые  необязательные мы обозначаем как:
    loct['must'].loc[little_m_have[:]] = 2
    loct = loct.loc[loct['must'] > 0]

    print(loct['must'].value_counts())
    loct = loct.rename(columns={m: 'volume'})
    loct['time_from'] = 480
    loct['time_to'] = 1200
    loct['time_spent'] = 10

    OptimusG.location = loct
    OptimusG.num_vehic = 5
    OptimusG.fixed = 1000_000
    # сама функция рассчета маршрутов
    route = OptimusG.vrp()

    routes += [route]

    sum_routes = flat([i['uid'] for i in route])
    sum_routes = [i for i in sum_routes if i != 0]

    print('Len sum_rout', len(sum_routes))
    print('Must ', len(set(little_bb_init)-set(sum_routes)))

    used += sum_routes

    da_inc = pd.concat([da_inc,table_incomes[[m]]], axis=1)


    # записываем в новый день
    table_incomes[m].loc[sum_routes] = 0
    table_incomes[n1] += table_incomes[m]




 Count 0
остаток на 31.08.2022 (входящий)


C:\Users\User\AppData\Local\Temp/ipykernel_11980/1355391931.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loct['must'] = (loct[m] + loct['90%'] > 1000).astype(int)
C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


2    1530
1      20
Name: must, dtype: int64


Формирование отчета

In [ ]:
# мой внутренний отчет
da_inc.to_excel(f'/root/proverka_inc.xlsx')      

# фаил остатков
table_incomes.to_excel(f'/root/table_inc.xlsx')

# маршрутов
with open(f'/root/dahun_pao.json', 'w') as outfile:
    json.dump(routes, outfile)

# сдвигаем колонки в фаиле тк мы считали на день в день
cols = [i.split(' ')[0] for i in table_incomes.columns[1:-1]]
table_incomes = table_incomes.iloc[:, :-2]
table_incomes.columns = cols
table_incomes = (table_incomes*1000).astype(int)


# 
d_start = pd.DataFrame()
for k, route in enumerate(routes):
    columns = ['uid', 'dist', 'time', 'volume', 'must']
    d = pd.DataFrame(flat([[list(ii)+[k] for ii in zip(*[j[i] for i in columns])]
                           for k, j in enumerate(route)]), columns=columns+['n'])
    d['date'] = table_incomes.columns[k]
    d_start = pd.concat([d_start, d], axis=0)

d_start = d_start.rename(columns={'uid': 'TID'})
d_start['was_taken'] = d_start['volume'].map(lambda x: x * 0.1 if x > 1_000 else 100)
d_start.to_excel(f'/root/_.xlsx')


incass_fond = table_incomes.applymap(lambda x: x*(2/100/365))

# формирование фаила инкас издержек (+100)
tabl_inc_reset = table_incomes.stack().reset_index().iloc[:, :2]
tabl_inc_reset.columns = ['TID', 'date']
table_ras = d_start[['TID', 'date', 'was_taken']].merge(tabl_inc_reset, on=['TID', 'date'], how='right')
table_ras = pd.pivot_table(table_ras, index='TID', columns='date', values='was_taken')

# формирование фаила итого
itogo = pd.concat([pd.DataFrame(table_ras.sum()).T, pd.DataFrame(incass_fond.sum()).T,
                   pd.DataFrame(incass_fond.sum()).T], axis=0)
itogo.iloc[2] = 20000*5
itogo = pd.concat([itogo, pd.DataFrame(itogo.sum()).T], axis=0)
itogo.index = ['фондирование', 'инкассация',
               'стоимость броневиков', 'итого']

# формирование фаила маршрутов
d_start = d_start.loc[d_start['TID'] != 0]
d_start['дата-время прибытия'] = d_start.apply(lambda x: pd.to_datetime(x['date'])+datetime.timedelta(minutes=x['time']), axis=1)
d_start['дата-время отъезда'] = d_start.apply(lambda x: pd.to_datetime(x['date'])+datetime.timedelta(minutes=x['time']+10), axis=1)
d_start = d_start.rename(columns={'n': 'порядковый номер броневика',
                                  'TID': 'устройство'}).set_index(['порядковый номер броневика', 'date'])
d_start = d_start.drop(columns=['dist', 'volume', 'time', 'must', 'was_taken'])

final_otch = [['остатки на конец дня', table_incomes],
         ['стоимость фондирования', incass_fond],
         ['стоимость инкассации', table_ras],
         ['маршруты', d_start],
         ['итог', itogo]]

with pd.ExcelWriter("/root/отсчёт.xlsx") as writer: 
    for i in final_otch:
        i[1].to_excel(writer, sheet_name=i[0])


table_ras = table_ras.applymap(lambda x: 1 if x > 0 else 0)
row_686 = [k for k, i in enumerate(table_incomes.columns) if k % 14 == 0]

for i in range(len(row_686)-1):
    row_rae = list(table_ras.loc[table_ras.iloc[:, row_686[i]:row_686[i+1]].sum(axis=1) == 0].index)
    if len(row_rae)>10:
        print(row_686[i], row_686[i+1])
    else:
        print(row_686[i], row_686[i+1], row_rae)